# 1. Xử lý time encoder và decoder

In [5]:
import os
import re
import pandas as pd

# --- CẤU HÌNH ---
ROOT_DIR = 'logs_RCF'
OUTPUT_FILE = os.path.join(ROOT_DIR, 'Overall_time.xlsx')

# 1. Regex Encode: "encoded 100 frames in 4.16s"
REGEX_ENCODE = r"encoded \d+ frames in ([\d.]+)s"

# 2. Regex Decode: "Total Time:        4.347 sec."
# \s+ nghĩa là khớp với một hoặc nhiều khoảng trắng liên tiếp
REGEX_DECODE = r"Total Time:\s+([\d.]+)\s+sec"

# --- THỨ TỰ SẮP XẾP MONG MUỐN ---
CUSTOM_ORDER = [
    "motion",
    "nonroi_CQP",
    "nonroi_CRF",
    "saliency",
    # "yolov11",
    # "yolov11_fullresol",
    # "yolov11_old",
    # "yolov11_openvino",
    # "yolov11_openvino_fullresol",
    # "yolov5",
    # "yolov5_fullresol",
    # "yolov5_old",
    # "yolov5_openvino",
    # "yolov5_openvino_fullresol",
    # "yolov8",
    # "yolov8_fullresol",
    # "yolov8_old",
    # "yolov8_openvino",
    # "yolov8_openvino_fullresol",
    # "yolov9",
    # "yolov9_fullresol",
    # "yolov9_old",
    # "yolov9_openvino",
    # "yolov9_openvino_fullresol"
]

def generate_excel_report():
    print(f"--> Đang quét dữ liệu trong thư mục: {ROOT_DIR}...")
    
    records = []

    # --- 1. QUÉT FILE ---
    for root, dirs, files in os.walk(ROOT_DIR):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                try:
                    path_parts = os.path.normpath(file_path).split(os.sep)
                    if len(path_parts) < 3: continue
                    
                    qp_name = path_parts[-2]
                    method_name = path_parts[-3]
                    
                    if method_name == ROOT_DIR: continue
                except Exception:
                    continue

                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    
                    # Lấy Encode Time
                    match_enc = re.search(REGEX_ENCODE, content)
                    enc_time = float(match_enc.group(1)) if match_enc else None
                    
                    # Lấy Decode Time
                    match_dec = re.search(REGEX_DECODE, content)
                    dec_time = float(match_dec.group(1)) if match_dec else None

                    # Chỉ lưu nếu lấy được ít nhất 1 trong 2 thông số
                    if enc_time is not None or dec_time is not None:
                        records.append({
                            'Method': method_name,
                            'QP': qp_name,
                            'Encode_Time': enc_time * 10 if enc_time is not None else None,
                            'Decode_Time': dec_time * 10 if dec_time is not None else None
                        })

    if not records:
        print("Không tìm thấy dữ liệu nào!")
        return

    # --- 2. XỬ LÝ DATAFRAME ---
    df = pd.DataFrame(records)
    
    # Hàm phụ trợ để xử lý bảng (Encode riêng, Decode riêng)
    def create_pivot(value_column):
        # Tính trung bình
        df_avg = df.groupby(['Method', 'QP'])[value_column].mean().reset_index()
        # Xoay bảng
        pivot = df_avg.pivot(index='Method', columns='QP', values=value_column)
        
        # Sắp xếp hàng (Method) theo Custom Order
        available = [m for m in CUSTOM_ORDER if m in pivot.index]
        others = [m for m in pivot.index if m not in CUSTOM_ORDER]
        pivot = pivot.reindex(available + others)
        
        # Sắp xếp cột (QP) theo số
        try:
            sorted_cols = sorted(pivot.columns, key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else 0)
            pivot = pivot[sorted_cols]
        except:
            pass
            
        # --- CẬP NHẬT MỚI: TÍNH AVERAGE ---
        # axis=1 nghĩa là tính trung bình theo hàng ngang (qua các cột QP)
        pivot['Average'] = pivot.mean(axis=1)
            
        return pivot

    # Tạo 2 bảng riêng biệt
    df_encode_final = create_pivot('Encode_Time')
    df_decode_final = create_pivot('Decode_Time')

    # --- 3. XUẤT RA EXCEL (2 SHEETS) ---
    print(f"--> Đang lưu file Excel tại: {OUTPUT_FILE}")
    
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        df_encode_final.to_excel(writer, sheet_name='Encode Time (s)')
        df_decode_final.to_excel(writer, sheet_name='Decode Time (s)')
        
    print("--> Hoàn tất! File có 2 tab: 'Encode Time' và 'Decode Time'.")

if __name__ == "__main__":
    generate_excel_report()

--> Đang quét dữ liệu trong thư mục: logs_RCF...
--> Đang lưu file Excel tại: logs_RCF/Overall_time.xlsx
--> Hoàn tất! File có 2 tab: 'Encode Time' và 'Decode Time'.


# 2. Extract roi time

# 3. TÍNH TOÁN PSNR và BPP
tôi có các file gốc yuv là 
```
input_yuv/
    class_B/
        Cactus_1920x1080_100.yuv
        ....
```
và các file yuv đã compress lần lượt nằm trong 
```output_RCF/
    motion/
        qp42/
            BasketballDrive_1920x1080_100.yuv
            ...
        qp47/
            ...
        ...
    yolov9/
        ...
```

In [4]:
import os
import re
import math
import numpy as np
import pandas as pd

# --- CẤU HÌNH ---
INPUT_DIR = 'input_yuv'          # Thư mục chứa video gốc
OUTPUT_DIR = 'output_RCF'        # Thư mục chứa video đã nén/reconstruct
RESULT_FILE = 'report_psnr_bpp.xlsx'

# Đuôi file bitstream để tính BPP (nếu file nén của bạn là .h265 hay .bit thì sửa ở đây)
COMPRESSED_EXT = '.bin' 

# Regex để lấy thông tin từ tên file: Name_WxH_Frames.yuv
# Ví dụ: Cactus_1920x1080_100.yuv
REGEX_FILENAME = r"(.+)_(\d+)x(\d+)_(\d+)"

# --- THỨ TỰ SẮP XẾP METHOD (như yêu cầu cũ) ---
CUSTOM_ORDER = [
    "motion",
    "nonroi_CQP",
    "nonroi_CRF",
    "saliency",
    "yolov11",
    "yolov11_fullresol",
    "yolov11_old",
    "yolov11_openvino",
    "yolov11_openvino_fullresol",
    "yolov5",
    "yolov5_fullresol",
    "yolov5_old",
    "yolov5_openvino",
    "yolov5_openvino_fullresol",
    "yolov8",
    "yolov8_fullresol",
    "yolov8_old",
    "yolov8_openvino",
    "yolov8_openvino_fullresol",
    "yolov9",
    "yolov9_fullresol",
    "yolov9_old",
    "yolov9_openvino",
    "yolov9_openvino_fullresol"
]

def parse_filename(filename):
    """Lấy Resolution và Frame count từ tên file"""
    match = re.search(REGEX_FILENAME, filename)
    if match:
        name = match.group(1)
        width = int(match.group(2))
        height = int(match.group(3))
        frames = int(match.group(4))
        return name, width, height, frames
    return None, 0, 0, 0

def calculate_y_psnr(raw_path, rec_path, width, height, frames):
    """Tính PSNR (chỉ kênh Y - Luma) giữa 2 file YUV"""
    # YUV420p: Frame size = W*H + W/2*H/2 * 2 = W*H * 1.5
    # Kênh Y chiếm W*H byte đầu tiên của mỗi frame
    y_size = width * height
    uv_size = (width // 2) * (height // 2) * 2
    frame_size = y_size + uv_size
    
    psnr_list = []
    
    try:
        with open(raw_path, 'rb') as f_raw, open(rec_path, 'rb') as f_rec:
            for _ in range(frames):
                # Đọc kênh Y
                raw_data = f_raw.read(y_size)
                rec_data = f_rec.read(y_size)
                
                # Nếu hết file sớm
                if not raw_data or not rec_data: 
                    break
                
                # Chuyển sang numpy array để tính toán nhanh
                raw_y = np.frombuffer(raw_data, dtype=np.uint8).astype(np.float64)
                rec_y = np.frombuffer(rec_data, dtype=np.uint8).astype(np.float64)
                
                mse = np.mean((raw_y - rec_y) ** 2)
                
                if mse == 0:
                    psnr_list.append(100.0) # Ảnh giống hệt nhau
                else:
                    psnr = 10.0 * math.log10((255.0 ** 2) / mse)
                    psnr_list.append(psnr)
                
                # Bỏ qua phần UV để nhảy sang frame tiếp theo
                f_raw.seek(uv_size, 1)
                f_rec.seek(uv_size, 1)
                
    except Exception as e:
        print(f"Lỗi khi đọc file {os.path.basename(raw_path)}: {e}")
        return None

    if not psnr_list:
        return None
        
    return np.mean(psnr_list)

def calculate_bpp(bin_path, width, height, frames):
    """Tính Bits Per Pixel từ file bitstream"""
    if not os.path.exists(bin_path):
        return None
        
    file_size_bytes = os.path.getsize(bin_path)
    total_bits = file_size_bytes * 8
    total_pixels = width * height * frames
    
    if total_pixels == 0: return 0
    return total_bits / total_pixels

def scan_files():
    # 1. Map file gốc: Tạo từ điển { 'TenFile_Goc': 'Duong_Dan_Full' }
    print("--> Đang quét thư mục Input...")
    raw_files_map = {}
    for root, dirs, files in os.walk(INPUT_DIR):
        for f in files:
            if f.endswith('.yuv'):
                # Key là tên file (ví dụ: Cactus_1920x1080_100.yuv)
                raw_files_map[f] = os.path.join(root, f)

    records = []
    print(f"--> Đang xử lý thư mục Output (tính PSNR & BPP)...")
    
    for root, dirs, files in os.walk(OUTPUT_DIR):
        for filename in files:
            if filename.endswith(".yuv"):
                # Cấu trúc output_RCF/Method/QP/File.yuv
                path_parts = os.path.normpath(os.path.join(root, filename)).split(os.sep)
                if len(path_parts) < 3: continue

                qp_name = path_parts[-2]      # vd: qp42
                method_name = path_parts[-3]  # vd: motion
                
                if method_name == OUTPUT_DIR: continue # Bỏ qua root

                # Tìm file gốc tương ứng
                if filename not in raw_files_map:
                    print(f"[Warn] Không tìm thấy file gốc cho: {filename}")
                    continue
                
                raw_path = raw_files_map[filename]
                rec_path = os.path.join(root, filename)
                
                # Parse thông số
                name_core, w, h, frames = parse_filename(filename)
                if w == 0 or h == 0: continue

                # --- TÍNH PSNR ---
                avg_psnr = calculate_y_psnr(raw_path, rec_path, w, h, frames)
                
                # --- TÍNH BPP ---
                # Giả định file bitstream (.bin) nằm cùng chỗ với file rec (.yuv)
                # Tên file bitstream thường giống file yuv nhưng khác đuôi
                bin_filename = filename.replace('.yuv', COMPRESSED_EXT)
                bin_path = os.path.join(root, bin_filename)
                
                bpp_val = calculate_bpp(bin_path, w, h, frames)
                
                if avg_psnr is not None:
                    print(f"[{method_name} - {qp_name}] {filename}: PSNR={avg_psnr:.2f}, BPP={bpp_val if bpp_val else 'N/A'}")
                    records.append({
                        'Method': method_name,
                        'QP': qp_name,
                        'PSNR': avg_psnr,
                        'BPP': bpp_val
                    })

    return pd.DataFrame(records)

def create_pivot(df, value_col):
    """Hàm tạo bảng Pivot và tính cột Average"""
    if df.empty: return pd.DataFrame()
    
    # Pivot
    pivot = df.pivot_table(index='Method', columns='QP', values=value_col, aggfunc='mean')
    
    # Sắp xếp Method
    available = [m for m in CUSTOM_ORDER if m in pivot.index]
    others = [m for m in pivot.index if m not in CUSTOM_ORDER]
    pivot = pivot.reindex(available + others)
    
    # Sắp xếp QP cột
    try:
        sorted_cols = sorted(pivot.columns, key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else 0)
        pivot = pivot[sorted_cols]
    except:
        pass
        
    # Tính Average theo hàng (trung bình các QP)
    pivot['Average'] = pivot.mean(axis=1)
    
    return pivot

def main():
    df = scan_files()
    
    if df.empty:
        print("Không có dữ liệu để xuất file.")
        return

    print(f"--> Đang xuất Excel: {RESULT_FILE}")
    
    # Tạo 2 bảng
    df_psnr = create_pivot(df, 'PSNR')
    df_bpp = create_pivot(df, 'BPP')
    
    with pd.ExcelWriter(RESULT_FILE, engine='openpyxl') as writer:
        df_psnr.to_excel(writer, sheet_name='Average PSNR (dB)')
        df_bpp.to_excel(writer, sheet_name='Average BPP')
        
    print("--> Hoàn tất!")

if __name__ == "__main__":
    main()

--> Đang quét thư mục Input...
--> Đang xử lý thư mục Output (tính PSNR & BPP)...


[yolov8 - qp47] BasketballDrive_1920x1080_100.yuv: PSNR=28.38, BPP=0.006162075617283951
[yolov8 - qp47] BQTerrace_1920x1080_100.yuv: PSNR=25.30, BPP=0.001887577160493827
[yolov8 - qp47] ParkScene_1920x1080_100.yuv: PSNR=26.53, BPP=0.00242662037037037
[yolov8 - qp47] Kimono_1920x1080_100.yuv: PSNR=28.64, BPP=0.0035142746913580246
[yolov8 - qp47] Cactus_1920x1080_100.yuv: PSNR=26.47, BPP=0.0038683256172839507
[yolov8 - qp42] BasketballDrive_1920x1080_100.yuv: PSNR=30.78, BPP=0.012004320987654322
[yolov8 - qp42] BQTerrace_1920x1080_100.yuv: PSNR=27.80, BPP=0.0032968364197530865
[yolov8 - qp42] ParkScene_1920x1080_100.yuv: PSNR=28.41, BPP=0.004983989197530864
[yolov8 - qp42] Kimono_1920x1080_100.yuv: PSNR=31.00, BPP=0.007100424382716049
[yolov8 - qp42] Cactus_1920x1080_100.yuv: PSNR=28.61, BPP=0.007674884259259259
[yolov8 - qp32] BasketballDrive_1920x1080_100.yuv: PSNR=35.43, BPP=0.04348707561728395
[yolov8 - qp32] BQTerrace_1920x1080_100.yuv: PSNR=32.55, BPP=0.020310262345679012
[yolov8 -

# 4. BD-rate

In [5]:
import pandas as pd
import numpy as np
import scipy.interpolate
import os

# --- CẤU HÌNH ---
ROOT_DIR = 'logs_RCF'
INPUT_FILE = os.path.join(ROOT_DIR, 'report_psnr_bpp.xlsx')

# Tên Method dùng làm mốc so sánh (Reference)
# Hãy đổi tên này trùng khớp với tên trong cột Method của file Excel
ANCHOR_METHOD = 'nonroi_CQP' 

def bd_rate(rate_anchor, psnr_anchor, rate_test, psnr_test):
    """
    Tính BD-Rate theo thuật toán VCEG-M33 (Bjøntegaard).
    Input:
        rate_anchor, psnr_anchor: List/Array BPP và PSNR của Anchor
        rate_test, psnr_test: List/Array BPP và PSNR của Test Method
    Output:
        Giá trị % thay đổi bitrate (BD-Rate).
    """
    
    # Chuyển sang numpy array và logarit cơ số 10 cho Rate
    ra = np.log10(rate_anchor)
    rt = np.log10(rate_test)
    pa = np.array(psnr_anchor)
    pt = np.array(psnr_test)

    # Tìm khoảng PSNR chung (Overlap range) để tích phân
    min_psnr = max(np.min(pa), np.min(pt))
    max_psnr = min(np.max(pa), np.max(pt))

    # Nếu không có khoảng chung hợp lệ
    if min_psnr >= max_psnr:
        return np.nan

    # Fit đa thức bậc 3: Log-Rate theo PSNR
    # p_anchor(x) = a + bx + cx^2 + dx^3
    p_anchor = np.polyfit(pa, ra, 3)
    p_test = np.polyfit(pt, rt, 3)

    # Tạo hàm đa thức từ hệ số
    poly_anchor = np.poly1d(p_anchor)
    poly_test = np.poly1d(p_test)

    # Tính tích phân xác định trong khoảng [min_psnr, max_psnr]
    # Hàm integ() của numpy trả về nguyên hàm, ta thay cận vào
    int_anchor = poly_anchor.integ()(max_psnr) - poly_anchor.integ()(min_psnr)
    int_test = poly_test.integ()(max_psnr) - poly_test.integ()(min_psnr)

    # Tính trung bình hiệu số log-rate
    avg_diff = (int_test - int_anchor) / (max_psnr - min_psnr)

    # Chuyển từ log space về phần trăm
    bd_rate_val = (np.power(10, avg_diff) - 1) * 100
    
    return bd_rate_val

def calculate_bd_rate_report():
    print(f"--> Đang đọc dữ liệu từ: {INPUT_FILE}")
    
    # 1. Đọc dữ liệu từ Excel
    try:
        df_psnr = pd.read_excel(INPUT_FILE, sheet_name='Average PSNR (dB)', index_col='Method')
        df_bpp = pd.read_excel(INPUT_FILE, sheet_name='Average BPP', index_col='Method')
    except Exception as e:
        print(f"Lỗi đọc file: {e}")
        return

    # 2. Lọc bỏ cột 'Average' nếu có, chỉ giữ lại các cột QP (qpXX)
    qp_cols = [c for c in df_psnr.columns if c != 'Average']
    
    # Kiểm tra xem Anchor có tồn tại không
    if ANCHOR_METHOD not in df_psnr.index:
        print(f"LỖI: Không tìm thấy method '{ANCHOR_METHOD}' trong file Excel để làm Anchor.")
        print(f"Các method hiện có: {list(df_psnr.index)}")
        return

    # Lấy dữ liệu của Anchor
    anchor_psnr = df_psnr.loc[ANCHOR_METHOD, qp_cols].values
    anchor_bpp = df_bpp.loc[ANCHOR_METHOD, qp_cols].values

    # Loại bỏ các giá trị NaN của Anchor (nếu có QP bị thiếu)
    valid_mask_a = ~np.isnan(anchor_psnr) & ~np.isnan(anchor_bpp)
    anchor_psnr = anchor_psnr[valid_mask_a]
    anchor_bpp = anchor_bpp[valid_mask_a]

    if len(anchor_psnr) < 4:
        print("Cảnh báo: Anchor có ít hơn 4 điểm dữ liệu (QP), kết quả BD-Rate có thể không chính xác.")

    results = []

    print(f"--> Đang tính toán BD-Rate (Anchor: {ANCHOR_METHOD})...")

    # 3. Duyệt qua từng method để tính BD-Rate
    for method in df_psnr.index:
        if method == ANCHOR_METHOD:
            results.append({'Method': method, 'BD-Rate (%)': 0.0})
            continue

        # Lấy dữ liệu Test Method
        test_psnr = df_psnr.loc[method, qp_cols].values
        test_bpp = df_bpp.loc[method, qp_cols].values

        # Lọc NaN
        valid_mask_t = ~np.isnan(test_psnr) & ~np.isnan(test_bpp)
        test_psnr = test_psnr[valid_mask_t]
        test_bpp = test_bpp[valid_mask_t]

        # Yêu cầu tối thiểu 4 điểm để fit curve bậc 3
        if len(test_psnr) < 4:
            print(f"Skip {method}: Không đủ 4 điểm dữ liệu.")
            results.append({'Method': method, 'BD-Rate (%)': 'N/A'})
            continue

        try:
            val = bd_rate(anchor_bpp, anchor_psnr, test_bpp, test_psnr)
            results.append({'Method': method, 'BD-Rate (%)': val})
        except Exception as e:
            print(f"Lỗi tính toán {method}: {e}")
            results.append({'Method': method, 'BD-Rate (%)': 'Error'})

    # 4. Lưu kết quả
    df_result = pd.DataFrame(results)
    
    # Format số đẹp (làm tròn 2 chữ số)
    # Lưu vào Sheet mới trong file cũ
    with pd.ExcelWriter(INPUT_FILE, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_result.to_excel(writer, sheet_name='BD-Rate Analysis', index=False)

    print(f"--> Hoàn tất! Đã thêm sheet 'BD-Rate Analysis' vào file {INPUT_FILE}")

if __name__ == "__main__":
    calculate_bd_rate_report()

--> Đang đọc dữ liệu từ: logs_RCF/report_psnr_bpp.xlsx
--> Đang tính toán BD-Rate (Anchor: nonroi_CQP)...
--> Hoàn tất! Đã thêm sheet 'BD-Rate Analysis' vào file logs_RCF/report_psnr_bpp.xlsx


In [6]:
import pandas as pd
import numpy as np
import scipy.interpolate
import os

# --- CẤU HÌNH ---
ROOT_DIR = 'logs_RCF'
INPUT_FILE = os.path.join(ROOT_DIR, 'report_psnr_bpp.xlsx')

# DANH SÁCH CÁC ANCHOR MUỐN SO SÁNH
# Lưu ý: Tên trong list này phải KHỚP CHÍNH XÁC (từng ký tự) với cột Method trong file Excel
ANCHOR_LIST = ['nonroi_CQP', 'nonroi_CRF'] 
# Nếu file của bạn tên là 'nonroi_CRF' thì sửa dòng trên thành: ['nonroi_CQP', 'nonroi_CRF']

def bd_rate(rate_anchor, psnr_anchor, rate_test, psnr_test):
    """
    Tính BD-Rate theo thuật toán VCEG-M33.
    """
    ra = np.log10(rate_anchor)
    rt = np.log10(rate_test)
    pa = np.array(psnr_anchor)
    pt = np.array(psnr_test)

    min_psnr = max(np.min(pa), np.min(pt))
    max_psnr = min(np.max(pa), np.max(pt))

    if min_psnr >= max_psnr:
        return np.nan

    try:
        # Fit đa thức bậc 3
        p_anchor = np.polyfit(pa, ra, 3)
        p_test = np.polyfit(pt, rt, 3)

        poly_anchor = np.poly1d(p_anchor)
        poly_test = np.poly1d(p_test)

        # Tích phân
        int_anchor = poly_anchor.integ()(max_psnr) - poly_anchor.integ()(min_psnr)
        int_test = poly_test.integ()(max_psnr) - poly_test.integ()(min_psnr)

        avg_diff = (int_test - int_anchor) / (max_psnr - min_psnr)
        bd_rate_val = (np.power(10, avg_diff) - 1) * 100
        return bd_rate_val
    except Exception:
        return np.nan

def calculate_bd_rate_report():
    print(f"--> Đang đọc dữ liệu từ: {INPUT_FILE}")
    
    try:
        df_psnr = pd.read_excel(INPUT_FILE, sheet_name='Average PSNR (dB)', index_col='Method')
        df_bpp = pd.read_excel(INPUT_FILE, sheet_name='Average BPP', index_col='Method')
    except Exception as e:
        print(f"Lỗi đọc file: {e}")
        return

    # Lọc lấy các cột QP (loại bỏ cột Average nếu có)
    qp_cols = [c for c in df_psnr.columns if str(c).lower() != 'average']
    
    # 1. CHUẨN BỊ DỮ LIỆU CỦA CÁC ANCHOR
    anchors_data = {}
    
    print(f"--> Đang tải dữ liệu Anchor: {ANCHOR_LIST}...")
    for anchor_name in ANCHOR_LIST:
        if anchor_name not in df_psnr.index:
            print(f"[CẢNH BÁO] Không tìm thấy method '{anchor_name}' trong file Excel. Bỏ qua.")
            anchors_data[anchor_name] = None
            continue
            
        a_psnr = df_psnr.loc[anchor_name, qp_cols].values
        a_bpp = df_bpp.loc[anchor_name, qp_cols].values
        
        # Lọc NaN
        mask = ~np.isnan(a_psnr) & ~np.isnan(a_bpp)
        anchors_data[anchor_name] = {
            'psnr': a_psnr[mask],
            'bpp': a_bpp[mask]
        }

    results = []

    # 2. TÍNH TOÁN CHO TỪNG METHOD
    print(f"--> Đang tính toán BD-Rate...")
    
    for method in df_psnr.index:
        # Tạo dictionary chứa kết quả cho dòng hiện tại
        row_result = {'Method': method}
        
        # Lấy dữ liệu Test
        t_psnr = df_psnr.loc[method, qp_cols].values
        t_bpp = df_bpp.loc[method, qp_cols].values
        mask_t = ~np.isnan(t_psnr) & ~np.isnan(t_bpp)
        t_psnr_clean = t_psnr[mask_t]
        t_bpp_clean = t_bpp[mask_t]

        # Duyệt qua từng Anchor để tính BD-Rate riêng biệt
        for anchor_name in ANCHOR_LIST:
            col_name = f'BD-Rate vs {anchor_name} (%)'
            
            # Nếu method hiện tại chính là anchor -> BD-Rate = 0
            if method == anchor_name:
                row_result[col_name] = 0.0
                continue
                
            # Nếu dữ liệu Anchor lỗi hoặc Test không đủ điểm
            if anchors_data.get(anchor_name) is None or len(t_psnr_clean) < 4:
                row_result[col_name] = 'N/A'
                continue

            # Lấy dữ liệu Anchor đã chuẩn bị
            a_psnr_clean = anchors_data[anchor_name]['psnr']
            a_bpp_clean = anchors_data[anchor_name]['bpp']
            
            if len(a_psnr_clean) < 4:
                row_result[col_name] = 'Anchor N/A'
                continue

            # Tính toán
            val = bd_rate(a_bpp_clean, a_psnr_clean, t_bpp_clean, t_psnr_clean)
            
            if np.isnan(val):
                row_result[col_name] = 'Range Error' # Không có khoảng overlap PSNR
            else:
                row_result[col_name] = val

        results.append(row_result)

    # 3. LƯU KẾT QUẢ
    df_result = pd.DataFrame(results)
    
    # Format lại index cho đẹp (cho cột Method ra đầu tiên)
    cols = ['Method'] + [f'BD-Rate vs {a} (%)' for a in ANCHOR_LIST if a in anchors_data]
    # Lọc lại phòng trường hợp có cột thừa do lỗi
    existing_cols = [c for c in cols if c in df_result.columns]
    df_result = df_result[existing_cols]

    with pd.ExcelWriter(INPUT_FILE, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_result.to_excel(writer, sheet_name='BD-Rate Comparison', index=False)

    print(f"--> Hoàn tất! Đã lưu sheet 'BD-Rate Comparison' vào file {INPUT_FILE}")
    print(f"--> Các cột được tạo: {[c for c in df_result.columns if c != 'Method']}")

if __name__ == "__main__":
    calculate_bd_rate_report()

--> Đang đọc dữ liệu từ: logs_RCF/report_psnr_bpp.xlsx
--> Đang tải dữ liệu Anchor: ['nonroi_CQP', 'nonroi_CRF']...
--> Đang tính toán BD-Rate...
--> Hoàn tất! Đã lưu sheet 'BD-Rate Comparison' vào file logs_RCF/report_psnr_bpp.xlsx
--> Các cột được tạo: ['BD-Rate vs nonroi_CQP (%)', 'BD-Rate vs nonroi_CRF (%)']


# 5. Draw vmaf

In [1]:
import os
import json
import numpy as np
import math

VMAF_ROOT = "./vmaf_results"
QPS = [22, 27, 32, 37, 42, 47]

def read_vmaf_mean(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    return data["pooled_metrics"]["vmaf"]["mean"]

table = {}

for method in sorted(os.listdir(VMAF_ROOT)):
    method_path = os.path.join(VMAF_ROOT, method)
    if not os.path.isdir(method_path):
        continue

    qp_values = {qp: [] for qp in QPS}

    for seq in os.listdir(method_path):
        seq_path = os.path.join(method_path, seq)
        if not os.path.isdir(seq_path):
            continue

        for qp in QPS:
            json_file = os.path.join(seq_path, f"vmaf_qp{qp}.json")
            if os.path.exists(json_file):
                qp_values[qp].append(read_vmaf_mean(json_file))

    qp_means = {}
    valid_qps = []

    for qp in QPS:
        if len(qp_values[qp]) > 0:
            mean_val = float(np.mean(qp_values[qp]))
            qp_means[qp] = mean_val
            valid_qps.append(mean_val)
        else:
            qp_means[qp] = None

    avg_vmaf = np.mean(valid_qps) if valid_qps else None
    table[method] = (qp_means, avg_vmaf)

# ===============================
# PRINT TABLE (SAFE FORMAT)
# ===============================
header = f"{'Method':22} " + " ".join([f"qp{qp:>4}" for qp in QPS]) + "  Average"
print(header)
print("-" * len(header))

for method, (qp_means, avg) in table.items():
    row = f"{method:22} "
    for qp in QPS:
        v = qp_means[qp]
        row += f"{v:6.2f} " if v is not None and not math.isnan(v) else "  --   "
    row += f"{avg:8.2f}" if avg is not None else "   --"
    print(row)


Method                 qp  22 qp  27 qp  32 qp  37 qp  42 qp  47  Average
-------------------------------------------------------------------------
motion                  94.82  87.85  76.18  60.69  43.83  29.28    65.44
motion_old              94.82  87.85  76.18  60.69  43.83  29.28    65.44
nonroi                  97.97  94.10  85.84  72.66  55.58  38.37    74.09
nonroi_CQP              98.42  94.76  86.48  72.74  54.93  36.81    74.02
nonroi_CQP_old          98.42  94.76  86.48  72.74  54.93  36.81    74.02
nonroi_CRF              95.98  90.58  80.89  66.31  48.61  32.44    69.14
nonroi_CRF_old          96.76  91.45  81.53  66.68  48.88  32.68    69.66
saliency                95.14  88.10  76.12  59.68  41.95  26.65    64.61
saliency_old            95.14  88.10  76.12  59.68  41.95  26.65    64.61
temp.hevc                --     --     --     --     --     --      --
yolov11                 95.63  89.44  78.44  63.20  46.14  31.33    67.36
yolov11_fullresol       95.81  89.87  79.